# Explore the data

In [9]:
import pandas as pd

In [10]:
# Read people and places csv files into dataframes
people = pd.read_csv('people.csv')
places = pd.read_csv('places.csv')

In [11]:
# Check dataframe sizes
print(f"people's shape:{people.shape}")
print(f"places'shape: {places.shape}")

people's shape:(10000, 4)
places'shape: (114, 3)


In [12]:
people.head()

,given_name,family_name,date_of_birth,place_of_birth
0,John,Williams,1842-09-30,Dumfries
1,Grace,Jeffery,1899-06-14,Kelso
2,Sean,Molnar,1982-11-01,Dromore
3,Lily,Doyle,1883-04-02,Hamilton
4,Edith,Styles,1879-07-24,Ballymoney


In [6]:
places.head()

,city,county,country
0,Aberdeen,Aberdeenshire,Scotland
1,Airdrie,Lanarkshire,Scotland
2,Alloa,Clackmannanshire,Scotland
3,Annan,Dumfriesshire,Scotland
4,Anstruther,Fife,Scotland


In [8]:
# Check duplicated rows
print(f"People table's duplicates: {people.duplicated().sum()}")
print(f"Places table's duplicates: {places.duplicated().sum()}")

People table's duplicates: 0
Places table's duplicates: 0


In [11]:
# Check duplicated rows
print(f"People table's nulls: {people.isnull().sum().sum()}")
print(f"Places table's nulls: {places.isnull().sum().sum()}")

People table's nulls: 0
Places table's nulls: 0


## Check the relationship between 2 tables

In [14]:
# outer join to include all data
joined_pdf = places.merge(people, how = 'outer', left_on = 'city', right_on = 'place_of_birth')
joined_pdf.head()

,city,county,country,given_name,family_name,date_of_birth,place_of_birth
0,Aberdeen,Aberdeenshire,Scotland,Jacob,Mountford,1875-11-07,Aberdeen
1,Aberdeen,Aberdeenshire,Scotland,Priscila,Hall,1902-11-07,Aberdeen
2,Aberdeen,Aberdeenshire,Scotland,Emily,Antell,1905-02-06,Aberdeen
3,Aberdeen,Aberdeenshire,Scotland,Stella,Edwards,1930-01-11,Aberdeen
4,Aberdeen,Aberdeenshire,Scotland,Victor,Leighton,1950-05-19,Aberdeen


In [15]:
joined_pdf.describe()

,city,county,country,given_name,family_name,date_of_birth,place_of_birth
count,10000,10000,10000,10000,10000,10000,10000
unique,114,38,2,1091,4680,9110,114
top,Campbeltown,Lanarkshire,Scotland,John,Smith,1902-04-07,Campbeltown
freq,114,783,8048,453,137,4,114


In [16]:
# Check NULLs to see if there's any unmatched data
joined_pdf.isnull().sum()

city              0
county            0
country           0
given_name        0
family_name       0
date_of_birth     0
place_of_birth    0
dtype: int64

The two table overlap 100%, there's no null value after joining two tables

# Explore `Places`

In [12]:
places.head()

,city,county,country
0,Aberdeen,Aberdeenshire,Scotland
1,Airdrie,Lanarkshire,Scotland
2,Alloa,Clackmannanshire,Scotland
3,Annan,Dumfriesshire,Scotland
4,Anstruther,Fife,Scotland


In [13]:
# Check unique values count
places.describe()

,city,county,country
count,114,114,114
unique,114,38,2
top,Aberdeen,Lanarkshire,Scotland
freq,1,9,92


## Normalize `Places`

### Create a `country` table

In [17]:
# Create a Country table
country_pdf = pd.DataFrame(places['country'].unique(), columns=['country'])
country_pdf

,country
0,Scotland
1,Northern Ireland


In [18]:
def assign_unique_id(unique_value, length_of_id):
    """Assign a unique id of type string to a value."""

    import hashlib

    m = hashlib.sha256()
    # remove whitespace and lowercase the string
    unique_value = str(unique_value).strip().lower()
    string = unique_value.encode('utf-8')
    m.update(string)
    return str(int(m.hexdigest(), 16))[0:length_of_id]

In [19]:
country_pdf['country_id'] = country_pdf['country'].apply(lambda x:assign_unique_id(x, 5))
country_pdf = country_pdf[['country_id', 'country']]
country_pdf

,country_id,country
0,93471,Scotland
1,19328,Northern Ireland


### Create a `city` table

In [20]:
city_pdf = places.copy().drop_duplicates()
city_pdf.head()

,city,county,country
0,Aberdeen,Aberdeenshire,Scotland
1,Airdrie,Lanarkshire,Scotland
2,Alloa,Clackmannanshire,Scotland
3,Annan,Dumfriesshire,Scotland
4,Anstruther,Fife,Scotland


In [22]:
# Check to see if there's any city name that belongs to more than 1 county or 1 country
city_pdf.groupby('city').agg({'county': 'nunique', 'country': 'nunique'}).reset_index().describe()

,county,country
count,114.0,114.0
mean,1.0,1.0
std,0.0,0.0
min,1.0,1.0
25%,1.0,1.0
50%,1.0,1.0
75%,1.0,1.0
max,1.0,1.0


In [23]:
city_pdf['city_id'] = city_pdf['city'].apply(lambda x : assign_unique_id(x, 8))
city_pdf.head()

,city,county,country,city_id
0,Aberdeen,Aberdeenshire,Scotland,85203947
1,Airdrie,Lanarkshire,Scotland,88361556
2,Alloa,Clackmannanshire,Scotland,71118086
3,Annan,Dumfriesshire,Scotland,81409057
4,Anstruther,Fife,Scotland,90346836


In [24]:
# Get country id from 'country' table
city_pdf = city_pdf.merge(country_pdf, how = 'left', left_on = 'country', right_on = 'country')
city_pdf = city_pdf[['city_id', 'city', 'county', 'country_id']]
city_pdf

,city_id,city,county,country_id
0,85203947,Aberdeen,Aberdeenshire,93471
1,88361556,Airdrie,Lanarkshire,93471
2,71118086,Alloa,Clackmannanshire,93471
3,81409057,Annan,Dumfriesshire,93471
4,90346836,Anstruther,Fife,93471
...,...,...,...,...
109,21688891,Thurso,Caithness,93471
110,65857060,Tillicoultry,Clackmannanshire,93471
111,46777161,West Calder,West Lothian,93471
112,68304277,Wick,Caithness,93471


## Normalize `People`

In [39]:
people_norm = people.copy()
people_norm.head()

,given_name,family_name,date_of_birth,place_of_birth
0,John,Williams,1842-09-30,Dumfries
1,Grace,Jeffery,1899-06-14,Kelso
2,Sean,Molnar,1982-11-01,Dromore
3,Lily,Doyle,1883-04-02,Hamilton
4,Edith,Styles,1879-07-24,Ballymoney


In [40]:
# Check to see if there's any duplicated records of name + dob
people_norm.groupby(['given_name', 'family_name', 'date_of_birth']).filter(lambda x: len(x) > 1)

,given_name,family_name,date_of_birth,place_of_birth


In [41]:
# Since every row is a unique person, I will add an unique id to each record
people_norm['person_id_raw'] = people_norm['given_name'].astype('str') + people_norm['family_name'].astype('str') + people_norm['date_of_birth'].astype('str') + people_norm['place_of_birth'].astype('str')
people_norm.head()

,given_name,family_name,date_of_birth,place_of_birth,person_id_raw
0,John,Williams,1842-09-30,Dumfries,JohnWilliams1842-09-30Dumfries
1,Grace,Jeffery,1899-06-14,Kelso,GraceJeffery1899-06-14Kelso
2,Sean,Molnar,1982-11-01,Dromore,SeanMolnar1982-11-01Dromore
3,Lily,Doyle,1883-04-02,Hamilton,LilyDoyle1883-04-02Hamilton
4,Edith,Styles,1879-07-24,Ballymoney,EdithStyles1879-07-24Ballymoney


In [42]:
people_norm['person_id'] = people_norm['person_id_raw'].apply(lambda x: assign_unique_id(x, 12))
people_norm.drop('person_id_raw', axis = 1, inplace=True)
people_norm

,given_name,family_name,date_of_birth,place_of_birth,person_id
0,John,Williams,1842-09-30,Dumfries,795671105626
1,Grace,Jeffery,1899-06-14,Kelso,760661749263
2,Sean,Molnar,1982-11-01,Dromore,184856746171
3,Lily,Doyle,1883-04-02,Hamilton,115632548783
4,Edith,Styles,1879-07-24,Ballymoney,688897386649
...,...,...,...,...,...
9995,Ethel,Wakeling,1921-09-22,Kirkcaldy,521438045178
9996,Robert,Hargreaves,1860-08-27,Haddington,105723823372
9997,Minnie,Walker,1888-05-25,Banff,190717867515
9998,Charles,Allgood,1866-02-11,Banff,105774353548


In [45]:
people_fact = people_norm.merge(city_pdf, how = 'left', left_on='place_of_birth', right_on = 'city')
people_fact = people_fact[['person_id', 'city_id', 'country_id']]
people_fact

,person_id,city_id,country_id
0,795671105626,86362822,93471
1,760661749263,77462074,93471
2,184856746171,12895620,19328
3,115632548783,31791753,93471
4,688897386649,62046352,19328
...,...,...,...
9995,521438045178,94374228,93471
9996,105723823372,39427481,93471
9997,190717867515,54133017,93471
9998,105774353548,54133017,93471


In [49]:
# Get number of persons by country
people_fact.groupby('country_id').agg({'person_id': 'nunique'}).reset_index().merge(country_pdf, how='inner', on = 'country_id')

,country_id,person_id,country
0,19328,1952,Northern Ireland
1,93471,8048,Scotland
